In [ ]:
import os, sys, subprocess, torch
%cd /work/projects/myproj
os.environ["MASTER_ADDR"]="127.0.0.1"
os.environ["MASTER_PORT"]="29500"
subprocess.run([sys.executable, "-m", "torch.distributed.run",
                f"--nproc_per_node={torch.cuda.device_count()}",
                "TrainRFDETR_Solo_SingleClass_Ucloud.py"], check=True)


In [ ]:
import os, sys, subprocess, torch, shutil
%cd /work/projects/myproj

# sanity + env
n = torch.cuda.device_count()
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "29500"

# Prefer console script if present, else use module
torchrun_path = shutil.which("torchrun")
if torchrun_path:
    cmd = [torchrun_path, f"--nproc_per_node={n}", "TrainRFDETR_Solo_SingleClass_Ucloud.py"]
else:
    # Works even when the console script isn't installed/visible
    # (PyTorch ≥1.9): torch.distributed.run; for very old, fallback to .launch
    try:
        import importlib
        importlib.import_module("torch.distributed.run")
        dist_module = "torch.distributed.run"
    except Exception:
        dist_module = "torch.distributed.launch"  # deprecated fallback
    cmd = [sys.executable, "-m", dist_module, f"--nproc_per_node={n}", "TrainRFDETR_Solo_SingleClass_Ucloud.py"]

print("Launching:", " ".join(cmd))
subprocess.run(cmd, check=True)